In [ ]:
import boto3

In [ ]:
import os
os.environ.setdefault('AWS_PROFILE', 'itvgenlogs')

In [ ]:
s3_client = boto3.client('s3')

In [1]:
def get_marker(file_name):
    marker_file_read = open('s3_marker.txt', 'r')
    marker = marker_file_read.read()
    return marker

In [2]:
def get_object_keys(Bucket, Prefix, Marker, MaxKeys):
    s3_objects = s3_client.list_objects(
        Bucket=Bucket,
        Prefix=Prefix,
        Marker=Marker,
        MaxKeys=MaxKeys
    ).get('Contents')
    if not s3_objects:
        return None
    s3_object_keys = [s3_object['Key'] for s3_object in s3_objects]   
    return s3_object_keys

In [ ]:
def get_data(s3_object_keys):
    data = []
    for s3_object_key in s3_object_keys:
        s3_object_contents = s3_client. \
            get_object(
                Bucket='itv-genlogs',
                Key=s3_object_key
            )['Body']. \
            read(). \
            decode('utf-8'). \
            splitlines()
        data += s3_object_contents
    return data

In [ ]:
def checkpoint(s3_object_keys):
    marker = s3_object_keys[-1]
    marker_file_write = open('s3_marker.txt', 'w')
    marker_file_write.write(marker)
    marker_file_write.close()
    return marker

In [ ]:
while True:
    marker = get_marker(file_name)

    s3_object_keys = get_object_keys(
        Bucket='itv-genlogs',
        Prefix='logs/year',
        Marker=marker,
        MaxKeys=10
    )
    if not s3_object_keys: break
    
    data = get_data(s3_object_keys)

    # Process data further and save to target

    marker = checkpoint(s3_object_keys)
 
    print(marker)